In [19]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [20]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier = planetary_computer.sign_inplace)

In [21]:
time_range = '2020-12-01/2020-12-31'
bbox = [-122.2751, 47.5469, -121.9613, 47.7458]

search = catalog.search(
    collections = ['landsat-c2-l2'],
    bbox = bbox,
    datetime = time_range
)

items = search.get_all_items()
len(items)

c:\Users\SABIO\Documents\GitHub\IE-University\myenv312\Lib\site-packages\pystac_client\item_search.py:896: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


8

In [22]:
items

In [23]:
area_of_interes = {
    'type': 'Polygon',
    'coordinates': [
        [
            [-122.2751, 47.5469],
            [-121.9613, 47.9613],
            [-121.9613, 47.9613],
            [-122.2751, 47.9613],
            [-122.2751, 47.5469],
        ]
    ]
}

search = catalog.search(
    collections = ['landsat-c2-l2'],
    intersects = area_of_interes,
    datetime = time_range
)

In [24]:
import geopandas 

df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs = "epsg:4326")
df.head()

,geometry,gsd,created,sci:doi,datetime,platform,proj:shape,description,instruments,eo:cloud_cover,proj:transform,view:off_nadir,landsat:wrs_row,landsat:scene_id,landsat:wrs_path,landsat:wrs_type,view:sun_azimuth,landsat:correction,view:sun_elevation,landsat:cloud_cover_land,landsat:collection_number,landsat:collection_category,proj:code
0,"POLYGON ((-122.72549 48.50884, -120.29248 48.0...",30,2022-05-06T18:04:17.126358Z,10.5066/P9OGBGM6,2020-12-29T18:55:56.738265Z,landsat-8,"[7881, 7781]",Landsat Collection 2 Level-2,"[oli, tirs]",100.00,"[30.0, 0.0, 471585.0, 0.0, -30.0, 5373315.0]",0,027,LC80460272020364LGN00,046,2,162.253231,L2SP,17.458298,100.00,02,T2,EPSG:32610
1,"POLYGON ((-124.52046 48.44245, -121.93932 48.0...",30,2022-05-06T17:25:29.626986Z,10.5066/P9C7I13B,2020-12-28T18:20:32.609164Z,landsat-7,"[7361, 8341]",Landsat Collection 2 Level-2,[etm+],31.00,"[30.0, 0.0, 333885.0, 0.0, -30.0, 5368515.0]",0,027,LE70470272020363EDC00,047,2,152.689113,L2SP,14.678880,32.00,02,T1,EPSG:32610
2,"POLYGON ((-122.96802 48.44547, -120.39024 48.0...",30,2022-05-06T18:01:04.319403Z,10.5066/P9C7I13B,2020-12-21T18:14:50.812768Z,landsat-7,"[7251, 8251]",Landsat Collection 2 Level-2,[etm+],25.00,"[30.0, 0.0, 452385.0, 0.0, -30.0, 5367315.0]",0,027,LE70460272020356EDC00,046,2,153.649177,L2SP,14.779612,24.00,02,T2,EPSG:32610
3,"POLYGON ((-124.27547 48.50831, -121.84167 48.0...",30,2022-05-06T17:46:22.246696Z,10.5066/P9OGBGM6,2020-12-20T19:02:09.878796Z,landsat-8,"[7971, 7861]",Landsat Collection 2 Level-2,"[oli, tirs]",100.00,"[30.0, 0.0, 353385.0, 0.0, -30.0, 5374215.0]",0,027,LC80470272020355LGN00,047,2,163.360118,L2SP,17.414441,100.00,02,T2,EPSG:32610
4,"POLYGON ((-122.72996 48.50858, -120.2969 48.07...",30,2022-05-06T18:04:16.935800Z,10.5066/P9OGBGM6,2020-12-13T18:56:00.096447Z,landsat-8,"[7881, 7781]",Landsat Collection 2 Level-2,"[oli, tirs]",98.73,"[30.0, 0.0, 471285.0, 0.0, -30.0, 5373315.0]",0,027,LC80460272020348LGN00,046,2,164.126188,L2SP,17.799744,98.64,02,T2,EPSG:32610


In [25]:
selected_item = min(items, key = lambda item: item.properties['eo:cloud_cover'])
print(selected_item)

<Item id=LC08_L2SP_047027_20201204_02_T1>


In [26]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                          │
│ ang              │ Angle Coefficients File                                              │
│ red              │ Red Band                                                             │
│ blue             │ Blue Band                                                            │
│ drad             │ Downwelled Radiance Band                                             │
│ emis             │ Emissivity Band                                                      │
│ emsd             │ Emissivity Standard Deviation Band                                   │
│ trad             │ Thermal Radiance Band                                                │
│ urad             │ Upwelled Radiance Band                                               │
│ atran            │ Atmospheric Transmittance Band                                       │
│ cdist            │ Cloud Distance Band                                                  │
│ green            │ Green Band                                                           │
│ nir08            │ Near Infrared Band 0.8                                               │
│ lwir11           │ Surface Temperature Band                                             │
│ swir16           │ Short-wave Infrared Band 1.6                                         │
│ swir22           │ Short-wave Infrared Band 2.2                                         │
│ coastal          │ Coastal/Aerosol Band                                                 │
│ mtl.txt          │ Product Metadata File (txt)                                          │
│ mtl.xml          │ Product Metadata File (xml)                                          │
│ mtl.json         │ Product Metadata File (json)                                         │
│ qa_pixel         │ Pixel Quality Assessment Band                                        │
│ qa_radsat        │ Radiometric Saturation and Terrain Occlusion Quality Assessment Band │
│ qa_aerosol       │ Aerosol Quality Assessment Band                                      │
│ tilejson         │ TileJSON with default rendering                                      │
│ rendered_preview │ Rendered preview                                                     │
└──────────────────┴──────────────────────────────────────────────────────────────────────┘

In [ ]:
import ollama

table = rich.table.Table("Asset Key", "Description", "Simplified Description")

for asset_key, asset in selected_item.assets.items():
    description = asset.title

    response = ollama.chat(
        model="llama3:8b",
        messages=[
            {"role": "system", "content": "You are an AI that explains asset descriptions in simple terms."},
            {"role": "user", "content": f"Please describe the asset with this description in simple terms: {description}"}
        ]
    )

    simplified_description = response['message']['content']
    table.add_row(asset_key, description, simplified_description)

table

In [46]:
def simplify_asset_description(asset):
    description = asset.title
    response = ollama.chat(
        model = "llama3:8b",
        messages=[
            {"role": "system", "content": "You are an AI that explains asset descriptions in simple terms."},
            {"role": "user", "content": f"Please describe the asset with this description in simple terms: {description}"}
        ]
    )
    return response['message']['content']

In [47]:
dictionary = {
    "Asset Key": [asset_key for asset_key, asset in selected_item.assets.items()],
    "Description": [asset.title for asset_key, asset in selected_item.assets.items()],
    "Simplified Description": [simplify_asset_description(asset) for asset_key, asset in selected_item.assets.items()]
}

df =pd.DataFrame(dictionary)
df

,Asset Key,Description,Simplified Description
0,qa,Surface Temperature Quality Assessment Band,"Let's break it down!\n\nThe ""Surface Temperatu..."
1,ang,Angle Coefficients File,"A nice and technical one!\n\nIn simple terms, ..."
2,red,Red Band,"A ""Red Band"" is a type of classification or ra..."
3,blue,Blue Band,"The ""Blue Band"" sounds like a brand name for s..."
4,drad,Downwelled Radiance Band,"A technical term!\n\nIn simple terms, ""Downwel..."
5,emis,Emissivity Band,"Let me break it down for you!\n\nAn ""Emissivit..."
6,emsd,Emissivity Standard Deviation Band,"Let me break it down for you!\n\nThe ""Emissivi..."
7,trad,Thermal Radiance Band,"The ""Thermal Radiance Band"" is a type of senso..."
8,urad,Upwelled Radiance Band,"Let me break it down for you!\n\nThe ""Upwelled..."
9,atran,Atmospheric Transmittance Band,"Let me break it down for you!\n\nAn ""Atmospher..."


In [60]:
df.to_csv('assets_table.csv', index = False)

In [58]:
print(df[df["Asset Key"] == 'qa']['Simplified Description'].iloc[0])

Let's break it down!

The "Surface Temperature Quality Assessment Band" is a way to measure and rate how well a surface (like a road or a building) is doing in terms of temperature.

Think of it like a report card for the surface. It's assessed based on its performance, and that performance is represented by a band (or range) of scores.

Here are some simple explanations:

* The "Surface Temperature" part refers to how hot or cold the surface is.
* "Quality Assessment" means that experts have evaluated this temperature data to give it a grade.
* A "Band" is like a range or a category that puts the surface's performance into one of several groups.

When someone says they're looking at the Surface Temperature Quality Assessment Band, they're likely trying to understand how well a surface is handling its temperature. Is it too hot? Too cold? Just right?

In simple terms, this asset helps us see how good or bad a surface is doing in terms of temperature, and that can be important for thing